In [0]:
pip install pyspark

In [0]:
from pyspark import SparkContext

In [0]:
sc=SparkContext()

In [27]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.its.dal.ca/apache/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

tar: spark-2.4.0-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [0]:
from pyspark import SparkContext
sc=SparkContext()


In [0]:
import os

In [0]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [30]:
os.listdir('./sample_data')

['anscombe.json',
 'README.md',
 'mnist_train_small.csv',
 'california_housing_test.csv',
 'mnist_test.csv',
 'california_housing_train.csv']

In [0]:
file_loc = './sample_data/california_housing_train.csv'

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.read.csv(file_loc, inferSchema=True, header =True)

In [40]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [45]:
df.show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

In [0]:
%%writefile ex.txt
first line
second line 
third line
fourth line

In [0]:
pip install py4j

In [0]:
#RDD Object 
textFilek=sc.textFile("ex.txt")

In [0]:
textFilek.count()

In [0]:
textFilek.first()

In [0]:
#sample transformation using filter 
secfind=textFilek.filter(lambda line: 'second' in line)

In [0]:
secfind.collect()#performing action on transformation

In [0]:
secfind.count()

In [48]:
%%writefile ex2.txt
first
second
third 
fourth

Overwriting ex2.txt


In [0]:
text_rdd=sc.textFile("ex2.txt")

In [0]:
words=text_rdd.map(lambda line:line.split())#array of arrays of every line with splitted words


In [0]:
words.collect()#complete array of single line of word splitted

In [0]:
text_rdd.flatMap(lambda line:line.split()).collect()#list of words splitted

In [57]:
#key value pair RDD
%%writefile serv.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing serv.txt


In [0]:
serv=sc.textFile('serv.txt')

In [60]:
serv.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [62]:
serv.map(lambda line:line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [0]:
clean = serv.map(lambda line:line[1:] if line[0]=='#' else line).collect()

In [0]:
clean = clean.map(lambda line:line.split())

In [0]:
clean.collect()

In [0]:
clean.map(lambda lst:(lst[3],lst[-1])).collect()#taking column of state and amount

In [0]:
painrs=clean.map(lambda lst:(lst[3],lst[-1]))

In [0]:
rekey=pairs.reduceByKey(lambda a1,a2:float(a1)+float(a2))

In [0]:
rekey.collect()

In [0]:
#grabb state and amount
step1=clean.map(lambda lst:(lst[3],lst[-1]))
#reduce by key
step2=step1.reduceByKey(lambda a1,a2:float(a1)+float(a2))
#get rid of state amount title
step3=step2.filter(lambda x:not x[0]=='State')
#sort results by amount
step4=step3.sortBy(lambda stAmt:stAmt[1],ascending=False)
#action
step5=step4.collect()

In [67]:
#tuple unpacking
x=['ID','State','Amount']
def id_st_Amt(id_St):
  #unpack values
  (id,St,amt)=id_St
  return amt
print(id_st_Amt(x)) 

Amount
